In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from unidecode import unidecode
import re
import csv 
from nltk.corpus import stopwords
import torch
from gensim.models import Word2Vec
from tqdm import tqdm 
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
word2vec = Word2Vec.load("SG_300_25_20/SG_300_25_20.model").wv

In [4]:
html_tag = r"<\S*>"
invalid_characters = r"[^a-zA-Z0-9 ]"
percentage = r"[0-9]+%"
hashtag = r"#\S*"
numeric = r"[0-9]+"
at = r"@\S*"
address = r"\S+@\S+\.\S+"
link = r"(https?:\/\/|www.)\S+"

def preprocess(s):
    # Remove html tags.
    s = re.sub(html_tag, " ", s)

    # Substitute percents.
    s = re.sub(percentage, " procent ", s)

    # Substitute hashtags.
    s = re.sub(hashtag, " hashtag ", s)

    # Substitute at.
    s = re.sub(at, " entitate ", s)

    # Substitute numbers. 
    s = re.sub(numeric, " numar ", s)

    # Substitute addresses.
    s = re.sub(address, " adresa ", s)

    # Substitute links.
    s = re.sub(link, " link ", s)

    # Remove accents, diacritics.
    s = unidecode(s)

    # Keep only these characters
    s = re.sub(invalid_characters, " ", s)
    return s

In [5]:
romainian_stopwords = set(stopwords.words("romanian"))

def tokenize(s):
    tokens = s.split()
    tokens = list(filter(lambda x: x not in romainian_stopwords, tokens))
    tokens = [s.lower() for s in tokens]
    return tokens

In [6]:
# Converts a list of tokens to a vector.
# Only look at the first num_tokens tokens.
def vectorize(tokens, num_tokens=2):
    vectors = []
    for i in range(num_tokens):
        if i >= len(tokens):
            vectors.append(torch.zeros(word2vec.vector_size))
        else:
            if tokens[i] in word2vec:
                vectors.append(torch.tensor(word2vec[tokens[i]])) 
            else:
                vectors.append(torch.zeros(word2vec.vector_size))
    return torch.cat(vectors, dim=0)

In [7]:
def load_train():
    df = pd.read_csv('train.csv')
    df = df.fillna("")

    contents =  [vectorize(tokenize(preprocess(x)), num_tokens=10) for x in tqdm(df["content"])]
    titles = [vectorize(tokenize(preprocess(x)), num_tokens=20) for x in tqdm(df["title"])]
    labels = list(df["class"])

    x = []
    for i in range(len(contents)):
        vec = torch.concat([contents[i], titles[i]], dim=0)
        x.append(vec)

    return x , labels 

In [8]:
x, labels = load_train() 
train_x, test_x, train_labels, test_labels = train_test_split(x, labels, test_size=0.2, shuffle=True)

100%|██████████| 70575/70575 [00:19<00:00, 3613.16it/s]


In [9]:
class MyDataset(Dataset):
    def __init__(self, x, labels):
        self.x = x
        self.labels = labels
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index], 1 if self.labels[index] == True else 0

NUM_FEATURES = train_x[0].shape[0]
train_dataset = MyDataset(train_x, train_labels)
test_dataset = MyDataset(test_x, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [10]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(NUM_FEATURES, 1000)
        self.layer2 = nn.Linear(1000, 500)
        self.layer3 = nn.Linear(500, 2)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.layer1(x)
        x = self.relu(x)

        x = self.layer2(x)
        x = self.relu(x)

        x = self.layer3(x)
        x = self.softmax(x)

        return x

    def predict(self, x):
        x = self.forward(x)
        return torch.argmax(x, dim=1)

In [49]:
loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor([3, 1.5]))
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [50]:
def train_one_epoch():
    model.train(True)
    running_loss = 0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in tqdm(enumerate(train_dataloader)):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()

        if i % 40 == 39:
            print(f"Running loss: {running_loss/(i+1)}") 

    return running_loss/(i+1)

In [51]:
def test_model():
    model.eval()

    with torch.no_grad():
        running_vloss = 0.0
        correct = 0
        total = 0

        for i, vdata in enumerate(test_dataloader):
            x , labels = vdata
            outputs = model(x)

            vloss = loss_fn(outputs, labels)
            running_vloss += vloss.item()

            predictions = model.predict(x) 
            correct += torch.sum(predictions == labels).item()
            total += len(labels)

        avg_vloss = running_vloss / (i + 1)

        return avg_vloss, correct/total

In [52]:
EPOCHS = 30 

for epoch in range(EPOCHS):
    train_loss = train_one_epoch()
    test_loss, test_accuracy = test_model()

    print(f"Train loss: {train_loss}, Test loss: {test_loss}, Test accuracy: {test_accuracy}")


41it [00:06,  6.05it/s]

Running loss: 0.442437706142664


81it [00:12,  5.88it/s]

Running loss: 0.4254419207572937


121it [00:19,  5.64it/s]

...

Running loss: 0.3779719527810812


120it [00:36,  3.42it/s]

Running loss: 0.3794906072318554


150it [00:46,  3.25it/s]


KeyboardInterrupt: 

In [48]:
print(test_model())

(0.36253542612705913, 0.9489904357066951)


In [40]:
def load_test():
    df = pd.read_csv('test.csv')
    df = df.fillna("")

    contents =  [vectorize(tokenize(preprocess(x)), num_tokens=10) for x in tqdm(df["content"])]
    titles = [vectorize(tokenize(preprocess(x)), num_tokens=20) for x in tqdm(df["title"])]

    x = []
    for i in range(len(contents)):
        vec = torch.concat([contents[i], titles[i]], dim=0)
        x.append(vec)

    return x 

In [41]:
x_test = load_test() 

100%|██████████| 36669/36669 [00:11<00:00, 3088.49it/s]


In [42]:

ids = []
predictions = []

for i in tqdm(range(len(x_test))):
    id = i
    prediction = model.predict(torch.unsqueeze(x_test[i], 0)).item()

    ids.append(id)
    predictions.append(prediction)


df = pd.DataFrame(data={"id":ids, "class": predictions})
print(df)

100%|██████████| 36669/36669 [02:30<00:00, 243.12it/s]


          id  class
0          0      0
1          1      0
2          2      0
3          3      0
4          4      0
...      ...    ...
36664  36664      0
36665  36665      0
36666  36666      0
36667  36667      1
36668  36668      0

[36669 rows x 2 columns]


In [43]:
df.to_csv("1.csv", index=False)